> 一个装饰器就是一个函数，它接受一个函数作为参数并返回一个新的函数

不带参数的装饰器（默认参数func，参见cookbook9.1）

In [1]:
def now():
    print('运行now')

In [2]:
now.__name__

'now'

wrapper是now的包裹函数，log函数负责打包流程

In [3]:
def log(func): # 接收被装饰函数作为参数
    print('log runned')
    def wrapper(*args, **kw): # 接受任意参数，这里和now的接受参数相同即可
        print '运行包裹函数'
        return func(*args, **kw) # 返回后续步骤，继续调用原函数;当然也可以拦截原函数的后续执行
    return wrapper # 返回包装函数

把@log放到now()函数的定义处，相当于执行了语句：`now = log(now)`

In [4]:
@log
def now():
    print('运行now')

log runned


如上，在函数定义阶段，log就运行了，将包装函数包裹好

In [5]:
now()

运行包裹函数
运行now


In [6]:
now.__name__

'wrapper'

带参数的装饰器：高一阶封装的打包函数

装饰器可以多层包裹，每个返回里层包裹函数（装饰器）

In [7]:
def log_func(msg): # 
    print('log_func has runned')
    def log(func): # func函数作为参数
        print('log has runned')
        def wrapper(*args, **kw): # now的参数
            print '运行包裹函数'
            return # 中断now的后续步骤
        return wrapper # 返回包装函数
    return log # 返回func函数，返回里一层的包裹函数

在函数定义阶段打好包

In [8]:
@log_func('wtf?')
def now():
    print('运行now')

log_func has runned
log has runned


In [9]:
now() # now函数原始流程被中断

运行包裹函数


In [10]:
now.__name__

'wrapper'

Python内置的functools.wraps负责`wrapper.__name__ = func.__name__`

在func参数函数里，包裹函数上方使用

In [11]:
from functools import wraps

def log_func2(msg): # 
    print('log_func has runned')
    def log(func): # func函数作为参数
        print('log has runned')
        @wraps(func) # 包裹函数，创建装饰器时保留函数元信息
        def wrapper(*args, **kw): # now的参数
            print '运行包裹函数'
            return # 中断now的后续步骤
        return wrapper # 返回包装函数
    return log # 返回func函数

@log_func2('wtf?')
def now():
    print('运行now')

log_func has runned
log has runned


In [12]:
now.__name__

'now'

作用到类上，扩充类的功能

In [1]:
def singleton(cls):
    instances = {}

    def getinstance():
        if cls not in instances:
            instances[cls] = cls()
        return instances[cls]

    return getinstance

In [ ]:
# cookbook 9.12 使用装饰器扩充类的功能
def log_getattribute(cls):    
    # Get the original implementation
    orig_getattribute = cls.__getattribute__

    # Make a new definition
    def new_getattribute(self, name):
        print('getting:', name)
        return orig_getattribute(self, name)

    # Attach to the class and return
    cls.__getattribute__ = new_getattribute
    return cls

# Example use
@log_getattribute
class A:
    def __init__(self,x):
        self.x = x
    def spam(self):
        pass

In [6]:
@singleton
class Ignore(object):
    def __str__(self):
        return 'Argument.ignore'
    #__repr__ = __str__

In [7]:
a = Ignore()
a

In [8]:
Ignore()